### Wie implementiert man eine KI?
Es gibt verschiedene Ansätze eine KI zu realisieren.
* statischer Ansatz: Support Vector Machines (SVM), Clustering-Verfahren
* evolutionärer Ansatz: genetische Algorithmen
* konnektionistischer Ansatz: künstliche neuronale Netze (KNN) <- aktuell State of the Art

### Was ist ein künstliches neuronales Netz?
Ein KNN lässt sich durch einen Graphen darstellen.
![Beispiel neuronales Netz](img/KNN.png)
* Input des Netzes nennen sich __Features__ (ganz selten Kovariate)
* Kanten haben ein Gewicht $w$ => trainierbare Parameter
* Knoten sind die Neuronen
* Neuronen enthalten eine Aktivierungsfunktion $f$
* KNNs approximieren im Grunde eine Funktion, die durch die Daten beschrieben wird

Seien $x \in \mathbb{R}$ ein Feature, $f_{A/B}: \mathbb{R} \to \mathbb{R}$ die Aktivierungsfunktionen der Neuronen und $w \in \mathbb{R}$ das Gewicht. Dann lässt dich das obige neuronale $k$ ausdrücken durch:
$k(x) = f_B(w f_A(x))$

![künstliches neuronales Netz](img/kuenstliche-neuronale-Netze.jpg)
Neuronale Netze sind in Schichten aufgeteilt:
* __Eingabeschicht (Input Layer):__ Größe des Input Layer wird durch die Daten bestimmt
* __Verbogene Schicht (Hidden Layer):__ Größe und Anzahl der Schichten sind variabel. Ab einer bestimmten Anzahl von Hidden Layers spricht man von Deep Learning.
* __Ausgabeschicht (Output Layer):__ Größe wird durch das Problem definiert.

### Wie lernt ein KNN?
Ein KNN lernt, indem es während des Trainings seine Netzgewichte $w_i$ anpasst. Während des Trainings erhält das Netz einen Datensatz mit den gewünschten Ergebnis (Labels). Dabei wird ein Datenpunkt aus dem Datensatz in das Netz reingegeben. Die Ausgabe des Netzes wird mit dann mit dem Label verglichen. Bei einer Abweichung werden die Netzgewichte so verändert, dass bei der nächsten Auswertung des Datenpunkts die Netzausgabe näher zum Label ist.
Ein kompletter Durchlauf des Trainingsdatensatzes nennt man __Epoche__.

#### Underfitting
Falls das neuronale Netz nicht genügend trainierbare Parameter hat, um das Problem aus dem Datensatz zu lernen, nennt man das Phänomen Underfitting. Das neuronale Netz muss in dem Fall vergrößert werden.

#### Overfitting
Das KNN performt während des Tranings sehr gut. In der Produktion ist die Fehlerrate ungewöhnlich hoch => Das KNN hat genug Kapazität, um die Beispiele aus dem Datensatz auswendig zu lernen.
Um das KNN zu zwingen, sich zu generalisieren, kann man das KNN verkleinern, den Datensatz vergrößern und/oder Regularisieren.

### Exkurs: Wie werden Bilder digital dargestellt?
#### Fall 1: Schwarz-Weiß-Bild
Ein Schwarz-Weiß-Bild lässt sich durch eine Matrix darstellen. Der Matrixeintrag $m_{xy} \in [0; 255]$ gibt an, wie hell/dunkel der Pixel an der Stelle $x, y$ ist.

#### Fall 2: Farbbilder
Farbbilder haben drei Farbkanäle (RGB). Jeder Farbkanal ist eine Matrix. Durch die Überlagerung der drei Farbkanäle erhält man das Farbbild.
![Tensor](img/Epsilontensor.svg.png)

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("img/color_image.jpg", 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)

In [ ]:
plt.imshow(img[:, :, 0], cmap="Reds")

In [ ]:
plt.imshow(img[:, :, 1], cmap="Blues")

In [ ]:
plt.imshow(img[:, :, 2], cmap="Greens")

### Convolutional Neural Networks (CNN)
Bisher können in den neuronalen Netzen nur Vektoren eingegeben werden. Bilder haben jedoch die Form einer Matrix oder eines Tensors. Man könnte die Bilder vektorisieren (flatten). Der räumliche Informationsgehalt eines Pixel wird jedoch dabei verringert.
Stattdessen haben sich Convolutional Neural Networks (CNN) als State-of-the-Art für Bilddaten etabliert.
![CNN](img/CNN.jpg)



#### Convolution-Operation
Bei der Convolution-Operation haben wir einen __Kernel__. Die Einträge im Kernel sind die trainierbaren Parameter des CNNs.
![convolution operation](img/convolution_operation.png)

Die Convolution-Operation verkleinert die Dimension des Bilds.


#### Pooling-Operation
Bei der Pooling-Operation werden die Informationen aus einem Teilbereich des Bilds weiter komprimiert.
![Max pooling operation](img/pooling.png)

In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers
import matplotlib.pyplot as plt

In [ ]:
(training_set, training_labels), (test_set, test_labels) = keras.datasets.mnist.load_data()
training_set = training_set.astype("float32") / 255.0
test_set = test_set.astype("float32") / 255.0

training_set = np.expand_dims(training_set, -1)
test_set = np.expand_dims(test_set, -1)

In [ ]:
training_set.shape

In [ ]:
plt.imshow(training_set[0], cmap='gray')

In [ ]:
training_labels[0]

#### Dokumentation zu den Keras Layers
__Convolution:__ \
https://keras.io/api/layers/convolution_layers/convolution2d/ \
https://keras.io/api/layers/convolution_layers/convolution3d/

__Pooling:__ \
https://keras.io/api/layers/pooling_layers/max_pooling2d/ \
https://keras.io/api/layers/pooling_layers/max_pooling3d/ \
https://keras.io/api/layers/pooling_layers/average_pooling2d/ \
https://keras.io/api/layers/pooling_layers/average_pooling3d/

__Fully Connected:__ \
https://keras.io/api/layers/core_layers/dense/

__Regularisierung:__ \
https://keras.io/api/layers/regularization_layers/dropout/

In [ ]:
input = keras.Input(shape=training_set.shape[1:])
hidden_layer = layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(input)
hidden_layer = layers.MaxPooling2D(pool_size=(2, 2))(hidden_layer)
hidden_layer = layers.Flatten()(hidden_layer)
hidden_layer = layers.Dropout(0.5)(hidden_layer)
output = layers.Dense(10, activation="softmax")(hidden_layer)

In [ ]:
img_classificator = keras.Model(inputs=input, outputs=output, name="conv_net_classification")
img_classificator.summary()

In [ ]:
img_classificator.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=["acc"])

In [ ]:
training_labels = keras.utils.to_categorical(training_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)
training_labels[0]

In [ ]:
history = img_classificator.fit(training_set, training_labels, batch_size=128, epochs=5, validation_split=0.2)

In [ ]:
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.show()

In [ ]:
score = img_classificator.evaluate(test_set, test_labels)
score[1]

In [ ]:
pred = img_classificator.predict(test_set[1:2])
pred

In [ ]:
np.argmax(pred)

In [ ]:
plt.imshow(test_set[1], cmap='gray')

### Weitere Datensätze
__Fashion MNIST__: https://keras.io/api/datasets/fashion_mnist/ \
__CIFAR10__: https://keras.io/api/datasets/cifar10/ \
__CIFAR100__: https://keras.io/api/datasets/cifar100/